## Cell 1: Environment Setup

In [1]:
from pathlib import Path
from datetime import datetime
import pandas as pd
import importlib

# Import and reload scanner module (ensures latest code changes are used)
import sp500_scanner
importlib.reload(sp500_scanner)
from sp500_scanner import get_sp500_tickers, scan_sp500, filter_buy_signals, create_excel_output, create_pdf_report, cleanup_old_scans

print("✓ Imports complete")
print(f"Ready to scan S&P 500 as of {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✓ Imports complete
Ready to scan S&P 500 as of 2026-01-05 21:01


## Cell 2: Run Full S&P 500 Scan

**This will take 5-10 minutes to complete.**

Parameters:
- `daily_bars=60` - 60 days of daily data
- `weekly_bars=52` - 52 weeks of weekly data  
- `concurrency=15` - 15 parallel threads (adjust if needed)

In [2]:
# Run the scan
start_time = datetime.now()
print(f"🔍 Starting S&P 500 scan at {start_time.strftime('%H:%M:%S')}...\n")

results_df = scan_sp500(daily_bars=60, weekly_bars=52, concurrency=15)

elapsed = (datetime.now() - start_time).total_seconds()
print(f"\n✓ Scan completed in {elapsed:.1f} seconds")

🔍 Starting S&P 500 scan at 21:02:02...

Fetching S&P 500 ticker list...
✓ Loaded 503 S&P 500 tickers

Found 503 S&P 500 stocks

🔍 Scanning 503 stocks for 'FULL HOLD + ADD' signals...
Parameters: 60 daily bars, 52 weekly bars, 15 threads
✓ [3/503] ABBV   -> FULL HOLD + ADD      $219.41
✓ [6/503] AFL    -> FULL HOLD + ADD      $112.32
✓ [13/503] ALB    -> FULL HOLD + ADD      $146.48
✓ [17/503] GOOGL  -> FULL HOLD + ADD      $315.94
✓ [20/503] AMZN   -> FULL HOLD + ADD      $232.93
✓ [21/503] ALL    -> FULL HOLD + ADD      $205.65
✓ [22/503] GOOG   -> FULL HOLD + ADD      $316.53
✓ [29/503] AXP    -> FULL HOLD + ADD      $380.74
✓ [30/503] AIG    -> FULL HOLD + ADD      $85.15
✓ [32/503] AMGN   -> FULL HOLD + ADD      $319.64
✓ [33/503] AME    -> FULL HOLD + ADD      $211.93
✓ [34/503] APO    -> FULL HOLD + ADD      $151.21
✓ [37/503] APP    -> FULL HOLD + ADD      $633.03
✓ [38/503] APH    -> FULL HOLD + ADD      $139.68
✓ [39/503] ADI    -> FULL HOLD + ADD      $277.83
✓ [40/503] AAPL 

## Cell 3: Filter and Display Results by Confluence

In [3]:
# Filter for FULL HOLD + ADD signals
buy_df = filter_buy_signals(results_df, 'FULL HOLD + ADD')

print(f"{'='*80}")
print(f"🎯 FULL HOLD + ADD SIGNALS: {len(buy_df)} stocks")
print(f"{'='*80}\n")

# Breakdown by confluence
print("📊 Breakdown by Confluence:")
confluence_counts = buy_df['confluence'].value_counts()
for conf, count in confluence_counts.items():
    print(f"  {conf:12s}: {count:3d} stocks")

# Save to workspace scanner_results folder
results_dir = Path.cwd() / 'scanner_results'
results_dir.mkdir(exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
excel_path = results_dir / f'sp500_analysis_{timestamp}.xlsx'
pdf_path = results_dir / f'scanner_report_{timestamp}.pdf'

# Generate Excel
create_excel_output(buy_df, excel_path)
print(f"\n✓ Excel saved to: {excel_path}")

# Generate PDF
print(f"\n📄 Creating PDF research document...")
create_pdf_report(buy_df, results_df, pdf_path, timestamp)
print(f"✓ PDF saved to: {pdf_path}")

# Cleanup old scans (keep 7 most recent, archive rest)
print("\n📁 Managing scan history...")
cleanup_old_scans(results_dir, max_files=7)

🎯 FULL HOLD + ADD SIGNALS: 202 stocks

📊 Breakdown by Confluence:
  WEAK        : 138 stocks
  EXTENDED    :  39 stocks
  BALANCED    :  25 stocks
✓ Excel file created: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2103.xlsx
  - All: 202 stocks
  - Balanced: 25 stocks
  - Extended: 39 stocks
  - Weak: 138 stocks

✓ Excel saved to: c:\workspace\my_script_project\scanner_results\sp500_analysis_20260105_2103.xlsx

📄 Creating PDF research document...
✓ PDF report created: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2103.pdf
  - 25 BALANCED stocks detailed
  - 39 EXTENDED stocks in watch list
✓ PDF saved to: c:\workspace\my_script_project\scanner_results\scanner_report_20260105_2103.pdf

📁 Managing scan history...


## Cell 4: Display BALANCED Stocks (Priority Buy List)

In [4]:
# Filter for BALANCED confluence only
balanced_df = buy_df[buy_df['confluence'] == 'BALANCED'].copy()
balanced_df = balanced_df.sort_values('ticker')

print(f"🎯 BALANCED STOCKS (Priority Buy Candidates): {len(balanced_df)}\n")

if not balanced_df.empty:
    # Display key columns
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200', 'w10', 'w200']
    available_cols = [col for col in display_cols if col in balanced_df.columns]
    
    print(balanced_df[available_cols].to_string(index=False))
    
    print(f"\n✅ These {len(balanced_df)} stocks have:")
    print("  ✓ FULL HOLD + ADD signal (Weekly P1 + Daily P1)")
    print("  ✓ BALANCED confluence (healthy technical setup)")
    print("  ✓ 'Enter on Dip' recommendation")
else:
    print("No BALANCED stocks found in this scan.")

🎯 BALANCED STOCKS (Priority Buy Candidates): 25

ticker  current_price recommendation        d50       d200        w10       w200
  AAPL       266.5830   Enter on Dip 272.927306 231.961426 273.241919 192.075811
   ALL       205.6500   Enter on Dip 204.372413 200.652723 207.069797 152.023959
  AMGN       319.6400   Enter on Dip 323.189772 294.274142 328.813226 261.673271
 BRK-B       500.2500   Enter on Dip 498.314832 497.789557 501.965158 395.114157
  CBOE       250.2600   Enter on Dip 251.678799 236.577324 253.331001 172.121504
   CEG       355.5400   Enter on Dip 360.477316 313.337904 354.762369 173.915560
  CINF       162.4716   Enter on Dip 162.205866 150.710768 163.947443 118.919189
  CSCO        75.7850   Enter on Dip  75.465330  67.039646  76.260978  51.473406
  CTRA        25.6900   Enter on Dip  25.748594  24.727640  26.217453  24.481066
    EW        84.2500   Enter on Dip  84.379800  78.079850  85.285001  81.407850
   EXE       105.9700   Enter on Dip 112.466108 106.296730 1

## Cell 5: Display EXTENDED Stocks (Watch List)

In [5]:
# Filter for EXTENDED confluence
extended_df = buy_df[buy_df['confluence'] == 'EXTENDED'].copy()
extended_df = extended_df.sort_values('ticker')

print(f"⏳ EXTENDED STOCKS (Wait for Pullback): {len(extended_df)}\n")

if not extended_df.empty:
    display_cols = ['ticker', 'current_price', 'recommendation', 'd50', 'd200']
    available_cols = [col for col in display_cols if col in extended_df.columns]
    
    # Show first 20
    print(extended_df[available_cols].head(20).to_string(index=False))
    
    if len(extended_df) > 20:
        print(f"\n... and {len(extended_df) - 20} more (see Excel file)")
    
    print(f"\n⚠️ These stocks have strong signals but are technically overbought.")
    print("  → Wait for pullback to support before entering")
else:
    print("No EXTENDED stocks found.")

⏳ EXTENDED STOCKS (Wait for Pullback): 39

ticker  current_price   recommendation        d50       d200
  ACGL        94.7600 Wait for Support  92.031200  91.305400
  AMZN       232.9300 Wait for Support 231.825800 217.215450
   APH       139.6800 Wait for Support 136.630449 107.053555
   AXP       380.7400 Wait for Support 365.804067 316.685991
   CAH       205.6750 Wait for Support 198.561030 162.192933
  CBRE       163.9350 Wait for Support 157.930700 145.898325
   CME       274.5550 Wait for Support 272.467766 269.330497
   CRH       128.4400 Wait for Support 119.777801 105.144963
  CTSH        81.8550 Wait for Support  77.404719  74.245567
   CVS        80.3700 Wait for Support  78.768200  70.199586
    DG       138.0300 Wait for Support 115.699600 105.662801
  EXPD       154.5700 Wait for Support 141.720193 121.840211
  EXPE       287.2900 Wait for Support 257.713495 202.829519
     F        13.4655 Wait for Support  13.165866  11.311818
  FSLR       270.4000 Wait for Support 260

## Cell 6: Compare with Current Holdings (stocks.txt)

In [6]:
from technical_analysis import parse_stocks_file

# Load current holdings
stocks_file = Path.cwd() / 'stocks.txt'
individual_tickers, baskets = parse_stocks_file(stocks_file)

# Find BALANCED stocks NOT in your portfolio
balanced_tickers = set(balanced_df['ticker'].tolist())
current_tickers = set(individual_tickers)
new_opportunities = balanced_tickers - current_tickers

print(f"📋 Current Portfolio: {len(current_tickers)} stocks")
print(f"🎯 BALANCED Opportunities: {len(balanced_tickers)} stocks")
print(f"\n✨ NEW BALANCED Stocks (not in your portfolio): {len(new_opportunities)}\n")

if new_opportunities:
    new_df = balanced_df[balanced_df['ticker'].isin(new_opportunities)].sort_values('ticker')
    display_cols = ['ticker', 'current_price', 'd50', 'd200', 'w10', 'w200']
    available_cols = [col for col in display_cols if col in new_df.columns]
    print(new_df[available_cols].to_string(index=False))
    print(f"\n💡 Consider adding these {len(new_opportunities)} stocks to stocks.txt for detailed analysis")
else:
    print("All BALANCED opportunities are already in your portfolio!")

# Check which current holdings are also BALANCED
holdings_balanced = balanced_tickers & current_tickers
print(f"\n✅ Current holdings that are BALANCED: {len(holdings_balanced)}")
if holdings_balanced:
    print(f"   {', '.join(sorted(holdings_balanced))}")

📋 Current Portfolio: 13 stocks
🎯 BALANCED Opportunities: 25 stocks

✨ NEW BALANCED Stocks (not in your portfolio): 24

ticker  current_price        d50       d200        w10       w200
  AAPL       266.5830 272.927306 231.961426 273.241919 192.075811
   ALL       205.6500 204.372413 200.652723 207.069797 152.023959
  AMGN       319.6400 323.189772 294.274142 328.813226 261.673271
 BRK-B       500.2500 498.314832 497.789557 501.965158 395.114157
  CBOE       250.2600 251.678799 236.577324 253.331001 172.121504
   CEG       355.5400 360.477316 313.337904 354.762369 173.915560
  CINF       162.4716 162.205866 150.710768 163.947443 118.919189
  CSCO        75.7850  75.465330  67.039646  76.260978  51.473406
  CTRA        25.6900  25.748594  24.727640  26.217453  24.481066
    EW        84.2500  84.379800  78.079850  85.285001  81.407850
   EXE       105.9700 112.466108 106.296730 113.516422  86.068663
   HSY       181.0450 179.939282 174.247023 182.083870 191.145470
   IBM       295.8600 3

## Cell 7: Overall Signal Distribution

In [7]:
print("📊 Complete S&P 500 Signal Distribution:\n")
print(results_df['signal'].value_counts().to_string())

print(f"\n\n{'='*80}")
print("Summary")
print(f"{'='*80}")
print(f"Total S&P 500 stocks analyzed: {len(results_df)}")
print(f"FULL HOLD + ADD signals: {len(buy_df)}")
print(f"  ├─ BALANCED (buy now): {len(buy_df[buy_df['confluence'] == 'BALANCED'])}")
print(f"  ├─ EXTENDED (watch): {len(buy_df[buy_df['confluence'] == 'EXTENDED'])}")
print(f"  └─ WEAK (skip): {len(buy_df[buy_df['confluence'] == 'WEAK'])}")
print(f"\nExcel file: {excel_path}")
print(f"PDF report: {pdf_path}")

📊 Complete S&P 500 Signal Distribution:

signal
FULL HOLD + ADD       202
HOLD MOST → REDUCE     76
FULL CASH / DEFEND     72
CASH                   56
REDUCE                 35
HOLD                   34
SCALE IN               20
LIGHT / CASH            7


Summary
Total S&P 500 stocks analyzed: 502
FULL HOLD + ADD signals: 202
  ├─ BALANCED (buy now): 25
  ├─ EXTENDED (watch): 39
  └─ WEAK (skip): 138


NameError: name 'output_path' is not defined